In [1]:
import numpy as np
import pandas as pd
import os
from data.buses import LTA
from data_proc import *

In [2]:
apiKey:str = "75cstockTLWN9wEcc9RE6Q=="

def to_df(data):
    data = pd.DataFrame.from_dict(data['value'])
    return data

lta = LTA(apiKey)



In [3]:
# Get df_bus_route
bus_route = lta.get_bus_routes()
df_bus_route = to_df(bus_route)
df_bus_route

,ServiceNo,Operator,Direction,StopSequence,BusStopCode,Distance,WD_FirstBus,WD_LastBus,SAT_FirstBus,SAT_LastBus,SUN_FirstBus,SUN_LastBus
0,10,SBST,1,1,75009,0.0,0500,2300,0500,2300,0500,2300
1,10,SBST,1,2,76059,0.6,0502,2302,0502,2302,0502,2302
2,10,SBST,1,3,76069,1.1,0504,2304,0504,2304,0503,2304
3,10,SBST,1,4,96289,2.3,0508,2308,0508,2309,0507,2308
4,10,SBST,1,5,96109,2.7,0509,2310,0509,2311,0508,2309
...,...,...,...,...,...,...,...,...,...,...,...,...
495,105,SBST,1,12,52189,5.0,0604,0018,0604,0018,0617,0016
496,105,SBST,1,13,52079,6.1,0607,0020,0606,0020,0620,0018
497,105,SBST,1,14,51099,7.3,0610,0023,0609,0023,0623,0020
498,105,SBST,1,15,40249,8.2,0612,0025,0611,0025,0625,0022


In [4]:
# Get df_bus_stops
# df_bus_stops
bus_stops = lta.get_bus_stops()
bus_stops = bus_stops['value']
nums =[500* i for i in range(1,11)]

for num in nums:
    bus_stops2 = lta.get_bus_stops2(num)  # modified method to generate dummy  * the api can call 500 record at once, so need to call multiple times to get all records
    bus_stops2 = bus_stops2['value']
    bus_stops.extend(bus_stops2)

print(len(bus_stops))
df_bus_stops =  pd.DataFrame.from_dict(bus_stops)
df_bus_stops

http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=1500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=2500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=3500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4000
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=4500
http://datamall2.mytransport.sg/ltaodataservice/BusStops?$skip=5000
5109


,BusStopCode,RoadName,Description,Latitude,Longitude
0,01012,Victoria St,Hotel Grand Pacific,1.296848,103.852536
1,01013,Victoria St,St. Joseph's Ch,1.297710,103.853225
2,01019,Victoria St,Bras Basah Cplx,1.296990,103.853022
3,01029,Nth Bridge Rd,Opp Natl Lib,1.296673,103.854414
4,01039,Nth Bridge Rd,Bugis Cube,1.298208,103.855491
...,...,...,...,...,...
5104,99139,Changi Village Rd,Blk 5,1.388195,103.987234
5105,99161,Nicoll Dr,Bef Changi Beach CP 3,1.390262,103.992957
5106,99171,Nicoll Dr,Changi Beach CP 2,1.391128,103.991021
5107,99181,Telok Paku Rd,Bef S'pore Aviation Ac,1.387754,103.988503


In [5]:
# Path to your CSV file
csv_file_path = 'data/transport_node_bus_202402.csv'

# Read the CSV file into a DataFrame
df_taps = pd.read_csv(csv_file_path)

# Preprocess df_taps
df_taps = preprocess_df_taps(df_taps)

df_taps

,YEAR_MONTH,PT_CODE,TOTAL_TAP_IN_VOLUME,TOTAL_TAP_OUT_VOLUME,TOTAL_TAP_VOLUME
0,2024-02,1012,28041,38209,66250
1,2024-02,1013,22187,16891,39078
2,2024-02,1019,16367,25547,41914
3,2024-02,1029,34387,22084,56471
4,2024-02,1039,58100,50405,108505
...,...,...,...,...,...
5089,2024-02,99139,36,19201,19237
5090,2024-02,99161,146,894,1040
5091,2024-02,99171,799,4034,4833
5092,2024-02,99181,13473,13863,27336


In [6]:
# Merge df_bus_route, df_bus_stops, and df_taps

df_taps['PT_CODE'].astype(int)
df_route_stops = df_bus_route.merge(df_bus_stops, left_on='BusStopCode', right_on='BusStopCode')
df_route_stops = df_route_stops[['ServiceNo','Direction','BusStopCode','Latitude','Longitude']]
df_route_stops['BusStopCode'] = df_route_stops['BusStopCode'].astype(int)
df_route_stops_taps = df_route_stops.merge(df_taps, left_on='BusStopCode', right_on='PT_CODE')
df_route_stops_taps = df_route_stops_taps[['ServiceNo','Direction','BusStopCode','Latitude','Longitude', 'TOTAL_TAP_VOLUME']]
df_route_stops_taps.rename(columns={'TOTAL_TAP_VOLUME': 'all_taps'}, inplace=True)
df_route_stops_taps

,ServiceNo,Direction,BusStopCode,Latitude,Longitude,all_taps
0,10,1,75009,1.354076,103.943391,2354906
1,10,1,76059,1.352962,103.941652,134683
2,10,1,76069,1.348753,103.942086,145510
3,10,1,96289,1.340055,103.948381,131149
4,10,1,96109,1.337371,103.950673,48687
...,...,...,...,...,...,...
495,105,1,52189,1.333185,103.846783,108713
496,105,1,52079,1.329315,103.843829,181859
497,105,1,51099,1.325882,103.834534,44009
498,105,1,40249,1.324925,103.827351,9819


In [7]:
# Test get_df_route_stops_taps
df_route_stops_taps = get_df_route_stops_taps(df_bus_route, df_bus_stops, df_taps)
df_route_stops_taps

,ServiceNo,Direction,BusStopCode,Latitude,Longitude,all_taps
0,100,1,1339,1.307746,103.864263,25767
1,100,2,1341,1.306476,103.864525,9672
2,100,1,1349,1.305686,103.865440,1608
3,100,2,1411,1.302113,103.862956,38750
4,100,1,1419,1.302020,103.863263,12822
...,...,...,...,...,...,...
495,10,1,94089,1.315363,103.942941,9926
496,10,2,96101,1.336385,103.951057,64736
497,10,1,96109,1.337371,103.950673,48687
498,10,2,96281,1.339714,103.948250,90795


In [8]:
df_bus_stops.to_csv('all_bus_stops.csv', index=False)

In [9]:
df_bus_route.to_csv('all_bus_routes.csv', index=False)